In [5]:
import tensorflow as tf
tf.reset_default_graph()

In [6]:
import numpy as np
wordsList = np.load('wordsList.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')

In [17]:
wordVectors.shape

(400000, 50)

In [7]:
numDimensions = 300
maxSeqLength = 200
batchSize = 24
lstmUnits = 64
numClasses = 53
iterations = 100000

In [8]:
labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [12]:
# sess = tf.InteractiveSession()
# saver = tf.train.Saver()


# saver = tf.train.import_meta_graph('models/pretrained_lstm.ckpt-90000.meta')
# saver.restore(sess,tf.train.latest_checkpoint('models'))

In [11]:
g = tf.get_default_graph()
g.get_operations()

[<tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'Placeholder_1' type=Placeholder>,
 <tf.Operation 'zeros' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'embedding_lookup/params_0' type=Const>,
 <tf.Operation 'embedding_lookup' type=Gather>,
 <tf.Operation 'DropoutWrapperInit/Const' type=Const>,
 <tf.Operation 'DropoutWrapperInit/Const_1' type=Const>,
 <tf.Operation 'DropoutWrapperInit/Const_2' type=Const>,
 <tf.Operation 'Rank' type=Const>,
 <tf.Operation 'range/start' type=Const>,
 <tf.Operation 'range/delta' type=Const>,
 <tf.Operation 'range' type=Range>,
 <tf.Operation 'concat/values_0' type=Const>,
 <tf.Operation 'concat/axis' type=Const>,
 <tf.Operation 'concat' type=ConcatV2>,
 <tf.Operation 'transpose' type=Transpose>,
 <tf.Operation 'rnn/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const' type=Const>,
 <tf.Operation 'rnn/DropoutWrapperZeroSta

In [13]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'Variable:0' shape=(24, 200, 300) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/kernel:0' shape=(114, 256) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(64, 53) dtype=float32_ref>,
 <tf.Variable 'Variable_2:0' shape=(53,) dtype=float32_ref>]

In [15]:
tf.global_variables()

[<tf.Variable 'Variable:0' shape=(24, 200, 300) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/kernel:0' shape=(114, 256) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(64, 53) dtype=float32_ref>,
 <tf.Variable 'Variable_2:0' shape=(53,) dtype=float32_ref>]

In [ ]:
import numpy as np
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model